In [1]:
import tensorflow as tf

Image is in the form `(width, height, Colour Channels)`

### Typical Architecture of a Convolutional Neural Network


- Input Image->Target Images we'd like to discover patterns in
- Input Layer->Takes in target images and preprocesses them for further layers - `input_shape=[batch_size,image_height, image_width, color_channels]`
- Convolutional Layer->Extracts/learns the most important features from target images -> Multiple, can create with `tf.keras.layers.ConvXD`
- Hidden Activation -> adds non-linearity to learned features(non-straight lines) -> Usually ReLU `tf.keras.activations.relu`
- Pooling layer-> Reduces the dimensionality of learned image features -> Average(`tf.keras.layers.AvgPool2D`) or Max(`tf.keras.layers.MaxPool2D`)
- Fully connected layer-> Further refines learned features from convolution layers -> `tf.keras.layers.Dense`
- Output layer->Takes learned features and outputs them in shape of targe labels-> `output_shape=[number_of_classes]
- Output activation->Adds non-linearities to output layer -> `tf.keras.activations.sigmoid`(binary classification) or `tf.keras.activations.softmax`

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
tf.random.set_seed(42)

train_datagen=ImageDataGenerator(rescale=1./255)
valid_datagen=ImageDataGenerator(rescale=1./255)

train_dir="pizza_steak/train/"
test_dir="pizza_steak/test/"

train_data=train_datagen.flow_from_directory(train_dir,
            batch_size=32,
            target_size=(224,224),
            class_mode="binary",
            seed=42
            )
valid_data=valid_datagen.flow_from_directory(test_dir,batch_size=32,
                                           target_size=(224,224),class_mode="binary",seed=42)

model_1=tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(filters=10,
                           kernel_size=3,
                           activation="relu",
                           input_shape=(224,224,3)),
    tf.keras.layers.Conv2D(10,3,activation="relu"),
    tf.keras.layers.MaxPool2D(pool_size=2,padding="valid"),
    tf.keras.layers.Conv2D(10,3,activation="relu"), 
    tf.keras.layers.Conv2D(10,3,activation="relu"),
    tf.keras.layers.MaxPool2D(2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1,activation="sigmoid")
])
model_1.compile(loss='binary_crossentropy',
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])
history_1=model_1.fit(train_data,
                      epochs=5,
                      steps_per_epoch=len(train_data),
                      validation_data=valid_data,
                      validation_steps=len(valid_data))

Found 1500 images belonging to 2 classes.
Found 500 images belonging to 2 classes.
Epoch 1/5
47/47 [==============================] - 19s 392ms/step - loss: 0.6011 - accuracy: 0.6560 - val_loss: 0.4451 - val_accuracy: 0.8040
Epoch 2/5
47/47 [==============================] - 17s 370ms/step - loss: 0.4607 - accuracy: 0.7860 - val_loss: 0.4066 - val_accuracy: 0.8340
Epoch 3/5
47/47 [==============================] - ETA: 0s - loss: 0.4328 - accuracy: 0.8053